In [3]:
# import required libraries for scraping and analysis
import urllib2
import json
import pandas as pd
from time import time
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import offsetbox
from sklearn.decomposition import TruncatedSVD
from sklearn import (manifold, datasets, decomposition, ensemble, lda,
                     random_projection)

RuntimeError: Python is not installed as a framework. The Mac OS X backend will not be able to function correctly if Python is not installed as a framework. See the Python documentation for more information on installing Python as a framework on Mac OS X. Please either reinstall Python as a framework, or try one of the other backends. If you are Working with Matplotlib in a virtual enviroment see 'Working with Matplotlib in Virtual environments' in the Matplotlib FAQ

In [4]:
# define endpoints from stats.nba.com that we wish to scrape
addressList = {
    "pullup_address": "http://stats.nba.com/js/data/sportvu/2014/pullUpShootData.json",
    "drives_address": "http://stats.nba.com/js/data/sportvu/2014/drivesData.json",
    "defense_address": "http://stats.nba.com/js/data/sportvu/2014/defenseData.json",
    "passing_address": "http://stats.nba.com/js/data/sportvu/2014/passingData.json",
    "touches_address": "http://stats.nba.com/js/data/sportvu/2014/touchesData.json",
    "speed_address": "http://stats.nba.com/js/data/sportvu/2014/speedData.json",
    "rebounding_address": "http://stats.nba.com/js/data/sportvu/2014/reboundingData.json",
    "catchshoot_address": "http://stats.nba.com/js/data/sportvu/2014/catchShootData.json",
    "shooting_address": "http://stats.nba.com/js/data/sportvu/2014/shootingData.json"
    }

# download and format SportVu data, data for each endpoint is placed in a dictionnary
sportsvu_data = {}
for key, val in addressList.items():
    print key
    url = val
    response = urllib2.urlopen(url)
    raw = response.read()
    data = json.loads(raw)
    headers = data['resultSets'][0]['headers']
    table = data['resultSets'][0]['rowSet']
    df = pd.DataFrame(table)
    df.columns = headers
    df = df.set_index(['PLAYER_ID', 'TEAM_ABBREVIATION'])
    df.columns = ['{}_{}'.format(key, x) for x in df.columns]
    sportsvu_data[key] = df

pullup_address
defense_address
rebounding_address
catchshoot_address
passing_address
drives_address
speed_address
shooting_address
touches_address


In [6]:
# concatenate all sportsvu data
sportvu = None
for suffix, input_df in sportsvu_data.items():
    print suffix
    if sportvu is None:
        sportvu = input_df
    else:
        sportvu = pd.merge(sportvu,
                           input_df,
                           how='inner',
                           left_index=True,
                           right_index=True)

pullup_address
rebounding_address
passing_address
touches_address
defense_address
catchshoot_address
drives_address
speed_address
shooting_address


In [9]:
# filter out dataframe to players who have had relevant playing time
df = sportvu
df = df[df.pullup_address_GP >= 41]
df = df[df.pullup_address_MIN >= 15]

# set index of final dataframe
df_final = df.T.drop_duplicates().T.reset_index()
df_final = df_final.set_index(['PLAYER_ID', 'TEAM_ABBREVIATION', 'pullup_address_PLAYER'])

# remove those columns because they are redundant
cols_to_remove = ['pullup_address_FIRST_NAME', 'pullup_address_LAST_NAME',
                  'pullup_address_GP', 'pullup_address_MIN']
df_final.drop(cols_to_remove, axis=1, inplace=True)

# remove duplicate columns
df_final = df.T.drop_duplicates().T
df_final

,,pullup_address_PLAYER,pullup_address_FIRST_NAME,pullup_address_LAST_NAME,pullup_address_GP,pullup_address_MIN,pullup_address_PTS,pullup_address_FGM,pullup_address_FGA,pullup_address_FG_PCT,pullup_address_FG3M,...,speed_address_DIST_48,speed_address_DIST_OFF,speed_address_DIST_DEF,speed_address_AV_SPD_OFF,shooting_address_PTS_CLOSE,shooting_address_FGP_CLOSE,shooting_address_FGA_DRIVE,shooting_address_FGA_CLOSE,shooting_address_EFG_PCT,shooting_address_CFGP
PLAYER_ID,TEAM_ABBREVIATION,,,,,,,,,,,,,,,,,,,,,
101106,GSW,Andrew Bogut,Andrew,Bogut,66,23.5,0.2,0.1,0.3,0.316,0,...,3,50.9,45.8,0,2.9,0.637,0.3,2.2,0.561,None
101107,CHA,Marvin Williams,Marvin,Williams,78,26.1,0.4,0.2,0.4,0.4,0,...,3.3,79.8,59.5,0,0.4,0.65,0.6,0.3,0.52,None
101108,LAC,Chris Paul,Chris,Paul,82,34.9,10.3,4.6,9.7,0.477,1.1,...,3.1,100.1,85.4,0,0.2,0.5,2.3,0.2,0.545,None
101112,ORL,Channing Frye,Channing,Frye,75,24.9,0.4,0.2,0.5,0.317,0,...,3,66.5,52.2,0,0.2,0.429,0.2,0.2,0.532,None
101114,BKN,Deron Williams,Deron,Williams,68,31.1,3.2,1.4,4.3,0.323,0.4,...,3.3,78.7,65.9,0,0.3,0.389,3.4,0.3,0.443,None
101123,PHX,Gerald Green,Gerald,Green,74,19.6,3.7,1.6,4.2,0.376,0.6,...,3.3,51.7,48.5,0,0.4,0.714,1.2,0.3,0.504,None
101127,BKN,Jarrett Jack,Jarrett,Jack,80,28.1,4.3,2.1,5.4,0.399,0.1,...,3.2,83.2,68.4,0,0.2,0.6,2.8,0.1,0.463,None
101133,IND,Ian Mahinmi,Ian,Mahinmi,61,18.8,0,0,0,0.333,0,...,3.2,40.4,36.6,0,2.3,0.607,0,1.8,0.552,None
101135,GSW,David Lee,David,Lee,48,18.5,0.1,0.1,0.3,0.2,0,...,3.2,31.3,28.3,0,2.8,0.621,0.5,1.8,0.519,None


In [25]:
# compute the correlation matrix between SportsVu metrics for all NBA players
# note: Pearson or Spearman showed little differences in results
df_final = df_final.convert_objects(convert_numeric=True)
# df_final = pd.to_numeric(df_final)
# df_final = df_final.astype(float)
corr = df_final.T.corr()
X = corr.as_matrix()

/Users/canzhiye/.virtualenvs/env1/lib/python2.7/site-packages/ipykernel/__main__.py:3: FutureWarning: convert_objects is deprecated.  Use the data-type specific converters pd.to_datetime, pd.to_timedelta and pd.to_numeric.
  app.launch_new_instance()


In [17]:
# perform t-SNE dimensionality reduction and print to file
# X_reduced = TruncatedSVD(n_components=10, random_state=0).fit_transform(X)
# tsne = manifold.TSNE(n_components=2, perplexity=40, verbose=2)
# X_tsne = tsne.fit_transform(X_reduced)

# df = pd.DataFrame(X_tsne)
# df['player'] = [x[2] for x in df_final.index]
# df['team'] = [x[1] for x in df_final.index]
# df.columns = ['x', 'y', 'player', 'team']
# df.to_csv('tsne_clusters.csv', index=False)